In [1]:
from cryptocmd import CmcScraper
import pandas as pd
import coinmarketcappy as cmc
from matplotlib import pyplot as plt
import seaborn as sns
import json
import requests
from bs4 import BeautifulSoup
import csv
import sys

In [ ]:
Debug = True #Make it False for fetching data first time
def CoinNames():
    """Gets ID's of all coins on cmc"""

    names = []
    response = requests.get("https://api.coinmarketcap.com/v1/ticker/?limit=0")
    respJSON = json.loads(response.text)
    for i in respJSON:
        if float(i['rank']) <=500:
            names.append(i['id'])
    return names

def gather(startdate, enddate, names):
    historicaldata = []
    counter = 1

    if len(names) == 0:
        names = CoinNames()

    for coin in names:
        r  = requests.get("https://coinmarketcap.com/currencies/{0}/historical-data/?start={1}&end={2}".format(coin, startdate, enddate))
        data = r.text
        soup = BeautifulSoup(data, "html.parser")
        table = soup.find('table', attrs={ "class" : "table"})

        #Add table header to list
        if len(historicaldata) == 0:
            headers = [header.text for header in table.find_all('th')]
            headers.insert(0, "Coin")

        for row in table.find_all('tr'):
            currentrow = [val.text for val in row.find_all('td')]
            if(len(currentrow) != 0):
                currentrow.insert(0, coin)
            historicaldata.append(currentrow)

        print("Coin Counter -> " + str(counter), end='\r')
        counter += 1
    return headers, historicaldata

if Debug is False:
    data = gather('20170701', '20180701', [])
    df = pd.DataFrame(list(data[1][1:]), columns=['Coin', 'Date', 'Open*', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])  
    df.to_csv('coindata.csv')

df = pd.read_csv('coindata.csv', thousands=',')
df.head()

In [ ]:
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Market Cap'] = df['Market Cap'].str.replace(',', '')
df['Market Cap'] = pd.to_numeric(df['Market Cap'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.head()